In [1]:



import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

we have to groupby the data for doing regration, then reset_index

## Groupby data set, make new columns for drawing

In [5]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,"Korea, South",1.0
2,2020-01-22,no,Kosovo,0.0
3,2020-01-22,no,Kuwait,0.0
4,2020-01-22,no,Kyrgyzstan,0.0


## Use test data to try and error

In [10]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-04-20')]
test_data.tail()

,date,state,country,confirmed
37322,2020-06-10,no,US,2000702.0
37548,2020-06-11,no,Germany,186691.0
37588,2020-06-11,no,US,2023590.0
37815,2020-06-12,no,Germany,187226.0
37856,2020-06-12,no,US,2048986.0


In [8]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-06-12,no,187226.0
US,2020-06-12,no,2048986.0


In [13]:
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    # every 3 days
    ''' Use a linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope   #approximation of doupling rate

def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [14]:
# droup "date", index is enough
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country       
no     Germany  23982            NaN
                24249            NaN
                24515      62.294061
                24780      70.294338
                25046      91.536840
                25312     112.903164
                25578     140.472457
                25845     148.285092
                26110     115.116625
                26375     104.285868
                26643     128.349094
                26909     167.535921
                27174     207.816845
                27440     279.484107
                27706     247.616282
                27972     166.275622
                28238     138.835878
                28504     139.648255
                28769     179.986624
                29037     265.319391
                29303     274.642705
                29568     267.092879
                29834     227.702584
                30100     266.129559
                30366     307.670485
                30633     274.967556
                

## deal with whole data set

In [19]:
#reset index= get previous index back, showed by level_2
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()
pd_DR_result.head()

,state,country,level_2,confirmed
0,Alberta,Canada,0,NaN
1,Alberta,Canada,435,NaN
2,Alberta,Canada,732,NaN
3,Alberta,Canada,966,NaN
4,Alberta,Canada,1232,NaN


In [20]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR','level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,435,NaN
2,Alberta,Canada,732,NaN
3,Alberta,Canada,966,NaN
4,Alberta,Canada,1232,NaN


In [21]:
#original data get a column called index 
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0.0
1,1,2020-01-22,no,"Korea, South",1.0
2,2,2020-01-22,no,Kosovo,0.0
3,3,2020-01-22,no,Kuwait,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0


In [22]:
#merge according to index of original data
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,1,2020-01-22,no,"Korea, South",1.0,NaN
2,2,2020-01-22,no,Kosovo,0.0,NaN
3,3,2020-01-22,no,Kuwait,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN


## Calculate DP rate of savgol_filtered data, make it smoother
1.filter the confirmed

In [23]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result

In [24]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0
1,1,2020-01-22,no,"Korea, South",1.0,NaN,0.8
2,2,2020-01-22,no,Kosovo,0.0,NaN,0.0
3,3,2020-01-22,no,Kuwait,0.0,NaN,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0


2. do DB rate

In [25]:

pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
38033,no,Zimbabwe,36772,21.510101
38034,no,Zimbabwe,37069,27.432099
38035,no,Zimbabwe,37304,26.976608
38036,no,Zimbabwe,37570,25.354497
38037,no,Zimbabwe,38037,25.553846


In [26]:
#merge into the big table
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
38033,38033,2020-06-12,no,Barbados,96.0,inf,96.8,79.666667
38034,38034,2020-06-12,no,Belarus,52520.0,71.252636,52561.0,67.394275
38035,38035,2020-06-12,no,Belgium,59819.0,477.597333,59820.0,490.940789
38036,38036,2020-06-12,no,Albania,1416.0,36.817778,1419.2,35.204082
38037,38037,2020-06-12,no,Zimbabwe,343.0,28.840580,345.2,25.553846


In [27]:
#calculate from larger than 100 confirmed!
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)

In [28]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)